# Notebook with Initial Experiment Visualisation Attempts

In [ ]:
# Jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
from solara.constants import PROJECT_PATH

In [ ]:
EXPERIMENT_NAME = "experiment_01_penalty_grid"
PLOT_DIR = PROJECT_PATH + "/figures/experiments/"

In [ ]:
import ray.tune

path = "./tmp/tune/PPO/experiment_state-2021-07-02_12-12-08.json"
path = "./tmp/tune/experiment_01_penalty_grid/" #experiment_state-2021-07-02_18-09-31.json"
path = "./tmp/tune/PPO/"#experiment_state-2021-07-02_12-12-08.json"


In [ ]:
import os

path = "./tmp/tune/experiment_01_penalty_grid/"
state_files = [filename for filename in os.listdir(path) if "experiment_state" in filename ]
last_state_file = sorted(state_files, reverse=True)[0]

analysis = ray.tune.ExperimentAnalysis(experiment_checkpoint_path=path + last_state_file)

In [ ]:
import pandas as pd
pd.options.display.max_columns=1000

In [ ]:
analysis.dataframe(metric="custom_metrics/cost_mean",mode="min")["custom_metrics/cost_mean"]#["config/env_config"][0]['general']['grid_charging']

In [ ]:
trials = analysis.fetch_trial_dataframes()

In [ ]:
trials = {key: trials[key] for key in sorted(trials.keys())} 

In [ ]:
def setup_matplotlib():
    plt.figure(figsize = (6, 4), 
               dpi = 100,
               tight_layout= True)
    sns.set_style("ticks", {"dashes":False})
    sns.set_context("paper")
    #sns.set_theme(style="whitegrid")
    #sns.set_palette()

In [ ]:
def plot_trials(trials, 
                necessary_cond=None,
                other_conditions=None, 
                experiment_name="default_experiment",
                plot_name = "plot_00_default",
                plot_dir = "./figures"):
    
    setup_matplotlib()
    
    other_conditions = list(other_conditions)
    
    for trial_name, trial_data in trials.items():
        if necessary_cond is None or necessary_cond[1] in trial_name:
            label = ""
            
            for i, (cond_label, condition) in enumerate(other_conditions):
                if condition in trial_name:
                    label += cond_label
                else: 
                    label += "no " + cond_label
                if i < len(other_conditions) - 1:
                    label += ", "
            
            label = label.capitalize()
            
            trace_len = 150
            x_values = np.arange(1,trace_len+1)
            ticks_gap = 25
            x_ticks = [1] + list(np.arange(ticks_gap,150+1, ticks_gap)) + [trace_len]
            trace = trial_data["custom_metrics/cost_mean"][0:trace_len]
            plt.plot(x_values,trace, label=label)
            
    plt.semilogy()
    plt.legend()
    plt.xlabel("Training iteration")
    plt.ylabel("Average cost per episode ($)")
    plt.xticks(x_ticks)
    
    plt.savefig(fname=plot_dir + experiment_name + "_" + plot_name + ".png")

In [ ]:
plot_trials(trials, 
            necessary_cond=["grid charging", "grid_charging=False"], 
            other_conditions=[["penalty","infeasible_control_penalty=True"]],
            experiment_name=EXPERIMENT_NAME,
            plot_dir=PLOT_DIR,
            plot_name="plot_01_penalty")

In [ ]:
plot_trials(trials, 
            necessary_cond=["penalty","infeasible_control_penalty=False"], 
            other_conditions=[["grid charging", "grid_charging=True"]],
            experiment_name=EXPERIMENT_NAME,
            plot_dir=PLOT_DIR,
            plot_name="plot_02_grid_charging_no_penalty")

In [ ]:
plot_trials(trials, 
            necessary_cond=["penalty","infeasible_control_penalty=True"], 
            other_conditions=[["grid charging", "grid_charging=True"]],
            experiment_name=EXPERIMENT_NAME,
            plot_dir=PLOT_DIR,
            plot_name="plot_03_grid_charging_with_penalty")

In [ ]:
other_conditions = [["penalty","infeasible_control_penalty=True"],
                    ["grid charging", "grid_charging=True"],]


plot_trials(trials, necessary_cond=None, other_conditions=other_conditions,
            experiment_name=EXPERIMENT_NAME,
            plot_dir=PLOT_DIR,
            plot_name="plot_04_all_trials")